# Entra Agent ID + Conditional Access for Hosted Agents

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **Copyright 2025 Ozgur Guler. All rights reserved.**

---

This notebook demonstrates how to:
1. Find your hosted agent's identity in Microsoft Entra
2. Compare shared vs distinct agent identities
3. Create a Conditional Access policy targeting the agent
4. Test and verify policy enforcement
5. Understand how capability hosts interact with agent identity

## Prerequisites

- Completed `../02-azd-deploy-hosted-agent` (working hosted agent)
- Microsoft Entra ID P1 or P2 license (for Conditional Access)
- Conditional Access Administrator role in your tenant
- Azure CLI installed and authenticated

---

## What is Microsoft Entra Agent ID?

**Microsoft Entra Agent ID** is a specialized identity framework for AI agents. It provides:

| Feature | Description |
|---------|-------------|
| **Agent Identity** | Unique, first-class identity for each agent (like user identities) |
| **Conditional Access** | Apply access policies to agents based on risk, location, etc. |
| **Identity Protection** | Detect and block risky agent behavior |
| **Governance** | Lifecycle management, owners, sponsors, expiration |

### Key Concept: Shared vs Distinct Identities

| Agent State | Identity Type | Scope |
|-------------|--------------|-------|
| **Unpublished** | Shared Project Identity | All unpublished agents share the same identity |
| **Published** | Distinct Agent Identity | Each published agent gets its own unique identity |

### Demo Scenario (Option B)

We'll demonstrate:
1. **Published agent** (`my-hosted-agent`) → Has DISTINCT identity → Can be blocked individually
2. **Unpublished agents** → Share PROJECT identity → Block one = block all

**Goal:** Create a CA policy that blocks ONLY the published agent while unpublished agents continue working.

---

## Section 1: Configuration

In [ ]:
import os
from dotenv import load_dotenv

# Load environment from parent directory
load_dotenv("../.env")

# Configuration
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID", "a20bc194-9787-44ee-9c7f-7c3130e651b6")
RESOURCE_GROUP = os.getenv("AZ_RESOURCE_GROUP", "rg-ozgurguler-7212")
FOUNDRY_ACCOUNT = os.getenv("FOUNDRY_ACCOUNT_NAME", "ozgurguler-7212-resource")
PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME", "ozgurguler-7212")
HOSTED_AGENT_NAME = "my-hosted-agent"

print(f"Subscription: {SUBSCRIPTION_ID}")
print(f"Resource Group: {RESOURCE_GROUP}")
print(f"Foundry Account: {FOUNDRY_ACCOUNT}")
print(f"Project: {PROJECT_NAME}")
print(f"Hosted Agent: {HOSTED_AGENT_NAME}")

---

## Section 2: Compare Shared vs Distinct Agent Identities

This is the key concept: **Unpublished agents share one identity, while published agents get their own.**

In [ ]:
# Get BOTH identities: Shared (for unpublished) and Distinct (for published)
import subprocess
import json
import requests

# 1. Get the SHARED project identity (used by ALL unpublished agents)
print("=" * 60)
print("1. SHARED PROJECT IDENTITY (Unpublished Agents)")
print("=" * 60)

# Get from project properties
result = subprocess.run([
    "az", "rest", "--method", "GET",
    "--url", f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_ACCOUNT}/projects/{PROJECT_NAME}?api-version=2025-04-01-preview"
], capture_output=True, text=True)

if result.returncode == 0:
    project_info = json.loads(result.stdout)
    agent_identity = project_info.get("properties", {}).get("agentIdentity", {})
    SHARED_AGENT_ID = agent_identity.get("agentIdentityId")
    SHARED_BLUEPRINT_ID = agent_identity.get("agentIdentityBlueprintId")
    
    print(f"Agent Identity ID:      {SHARED_AGENT_ID}")
    print(f"Blueprint ID:           {SHARED_BLUEPRINT_ID}")
    print()
    print("All unpublished agents (chatbot-gpt5nano, agent-jj25nd, echo-za9g)")
    print("share this SAME identity.")
else:
    print(f"Error: {result.stderr}")
    SHARED_AGENT_ID = None

In [ ]:
# 2. Get the DISTINCT identity for PUBLISHED agent (my-hosted-agent)
print("=" * 60)
print("2. DISTINCT IDENTITY (Published Agent: my-hosted-agent)")
print("=" * 60)

# Get published applications
result = subprocess.run([
    "az", "rest", "--method", "GET",
    "--url", f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_ACCOUNT}/projects/{PROJECT_NAME}/applications?api-version=2025-10-01-preview"
], capture_output=True, text=True)

PUBLISHED_AGENT_ID = None
if result.returncode == 0:
    apps = json.loads(result.stdout).get("value", [])
    for app in apps:
        if app.get("name") == "my-hosted-agent":
            instance_id = app.get("properties", {}).get("defaultInstanceIdentity", {})
            blueprint = app.get("properties", {}).get("agentIdentityBlueprint", {})
            
            PUBLISHED_AGENT_ID = instance_id.get("clientId")
            PUBLISHED_BLUEPRINT_ID = blueprint.get("clientId")
            
            print(f"Agent Instance ID:      {PUBLISHED_AGENT_ID}")
            print(f"Blueprint ID:           {PUBLISHED_BLUEPRINT_ID}")
            print()
            print("This is a UNIQUE identity, different from the shared project identity.")
            break
    else:
        print("Published agent 'my-hosted-agent' not found")
else:
    print(f"Error: {result.stderr}")

# Show the key difference
print()
print("=" * 60)
print("KEY INSIGHT:")
print("=" * 60)
print(f"Shared ID:    {SHARED_AGENT_ID}")
print(f"Published ID: {PUBLISHED_AGENT_ID}")
print()
if SHARED_AGENT_ID != PUBLISHED_AGENT_ID:
    print("✅ These are DIFFERENT identities!")
    print("   → CA policy on Published ID = blocks ONLY my-hosted-agent")
    print("   → Unpublished agents still work")
else:
    print("⚠️  IDs match - agent may not be properly published")

### View in Entra Admin Center

You can also view agent identities in the Microsoft Entra Admin Center:

1. Go to [Microsoft Entra Admin Center](https://entra.microsoft.com)
2. Navigate to **Entra ID > Agent ID > Agent identities**
3. Look for your agent in the list

Or view the service principal directly:

In [ ]:
# View the service principal in Entra
if AGENT_PRINCIPAL_ID:
    result = subprocess.run(
        ["az", "ad", "sp", "show", "--id", AGENT_PRINCIPAL_ID, "-o", "json"],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        sp_info = json.loads(result.stdout)
        print("Service Principal Details:")
        print(f"  Display Name: {sp_info.get('displayName')}")
        print(f"  App ID: {sp_info.get('appId')}")
        print(f"  Object ID: {sp_info.get('id')}")
        print(f"  Type: {sp_info.get('servicePrincipalType')}")
    else:
        print(f"Note: Could not retrieve SP details. Error: {result.stderr}")
        print("The managed identity may not be visible as a standard service principal.")

---

## Section 3: Create a Conditional Access Policy

### The Simplest Demo: Block Agent Access to a Specific Resource

We'll create a Conditional Access policy that:
1. Targets our agent identity
2. Blocks access to Azure OpenAI (the resource our agent uses)
3. Shows enforcement in sign-in logs

**Policy Logic:**
```
IF agent_identity = our_hosted_agent
AND target_resource = Azure Cognitive Services
THEN BLOCK
```

### Important: Creating CA Policies Requires Portal or Graph API

Conditional Access policies for agents are created via:
- **Microsoft Entra Admin Center** (recommended for first-time setup)
- **Microsoft Graph API** (for automation)

### Option A: Create Policy via Entra Admin Center (Manual Steps)

1. Go to [Microsoft Entra Admin Center](https://entra.microsoft.com)
2. Navigate to **Protection > Conditional Access > Policies**
3. Click **+ New policy**
4. Configure:

| Setting | Value |
|---------|-------|
| **Name** | `Block Hosted Agent - Demo` |
| **Assignments > Users, agents, or workload identities** | Select **Agents (Preview)** |
| **Include** | Select agent identities > Choose your agent's identity |
| **Target resources** | All resources (or select Azure Cognitive Services) |
| **Grant** | Block access |
| **Enable policy** | Report-only (for testing) or On |

5. Click **Create**

### Option B: Create Policy via Microsoft Graph API

For programmatic creation, use the Microsoft Graph API. First, get an access token:

In [ ]:
# Get access token for Microsoft Graph
result = subprocess.run(
    ["az", "account", "get-access-token", 
     "--resource", "https://graph.microsoft.com",
     "-o", "json"],
    capture_output=True, text=True
)

if result.returncode == 0:
    token_info = json.loads(result.stdout)
    GRAPH_TOKEN = token_info.get("accessToken")
    print("Got Microsoft Graph access token")
    print(f"Expires: {token_info.get('expiresOn')}")
else:
    print(f"Error getting token: {result.stderr}")
    GRAPH_TOKEN = None

In [ ]:
# List existing Conditional Access policies
import requests

if GRAPH_TOKEN:
    headers = {"Authorization": f"Bearer {GRAPH_TOKEN}"}
    
    resp = requests.get(
        "https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies",
        headers=headers
    )
    
    if resp.status_code == 200:
        policies = resp.json().get("value", [])
        print(f"Found {len(policies)} existing Conditional Access policies:")
        for p in policies[:10]:  # Show first 10
            state = p.get('state', 'unknown')
            print(f"  - {p.get('displayName')} ({state})")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
        print("\nNote: You need Conditional Access Administrator role to list policies.")

In [ ]:
# Create a Conditional Access policy targeting ONLY the PUBLISHED agent
# This will block my-hosted-agent but leave unpublished agents working!

POLICY_NAME = "Block Published Agent Only - Demo"

# Azure Cognitive Services app ID (this is what Azure OpenAI uses)
COGNITIVE_SERVICES_APP_ID = "fcc0e4a3-794f-4b44-a6a2-bf75d57a9644"

# Use the PUBLISHED agent's distinct identity (not the shared one!)
TARGET_IDENTITY = PUBLISHED_AGENT_ID if PUBLISHED_AGENT_ID else "066a09ef-e2ca-4a4d-80d5-914084673305"

policy_body = {
    "displayName": POLICY_NAME,
    "state": "enabledForReportingButNotEnforced",  # Report-only mode for testing
    "conditions": {
        "clientApplications": {
            "includeServicePrincipals": [TARGET_IDENTITY],
            "excludeServicePrincipals": []
        },
        "applications": {
            "includeApplications": [COGNITIVE_SERVICES_APP_ID],
            "excludeApplications": []
        }
    },
    "grantControls": {
        "operator": "OR",
        "builtInControls": ["block"]
    }
}

print("=" * 60)
print("CONDITIONAL ACCESS POLICY CONFIGURATION")
print("=" * 60)
print(f"Policy Name: {POLICY_NAME}")
print(f"Target Identity: {TARGET_IDENTITY}")
print(f"  (This is the PUBLISHED agent's distinct identity)")
print()
print("What this policy does:")
print("  ✅ Blocks: my-hosted-agent (published)")
print("  ✅ Allows: chatbot-gpt5nano, agent-jj25nd, echo-za9g (unpublished)")
print()
print("Policy JSON:")
print(json.dumps(policy_body, indent=2))

In [ ]:
# Actually create the policy (uncomment to execute)
# WARNING: This will create a real Conditional Access policy!

CREATE_POLICY = False  # Set to True to create the policy

if CREATE_POLICY and GRAPH_TOKEN and AGENT_PRINCIPAL_ID:
    headers = {
        "Authorization": f"Bearer {GRAPH_TOKEN}",
        "Content-Type": "application/json"
    }
    
    resp = requests.post(
        "https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies",
        headers=headers,
        json=policy_body
    )
    
    if resp.status_code == 201:
        created_policy = resp.json()
        print(f"Policy created successfully!")
        print(f"  ID: {created_policy.get('id')}")
        print(f"  Name: {created_policy.get('displayName')}")
        print(f"  State: {created_policy.get('state')}")
    else:
        print(f"Error creating policy: {resp.status_code}")
        print(resp.text)
else:
    print("Policy creation skipped (CREATE_POLICY = False)")
    print("\nTo create the policy:")
    print("1. Set CREATE_POLICY = True above")
    print("2. Re-run this cell")
    print("\nOr create manually in the Entra Admin Center (see Option A above)")

---

## Section 4: Test the Demo - Published vs Unpublished

After creating and enabling the CA policy, let's test both types of agents.

**Expected Results:**
- `my-hosted-agent` (published) → ❌ BLOCKED
- `chatbot-gpt5nano` (unpublished) → ✅ WORKS
- `agent-jj25nd` (unpublished) → ✅ WORKS

In [ ]:
# Setup: Connect to Foundry
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

PROJECT_ENDPOINT = f"https://{FOUNDRY_ACCOUNT}.services.ai.azure.com/api/projects/{PROJECT_NAME}"

credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
openai_client = client.get_openai_client()

print(f"Connected to: {PROJECT_ENDPOINT}")

# Define test agents
PUBLISHED_AGENTS = ["my-hosted-agent"]
UNPUBLISHED_AGENTS = ["chatbot-gpt5nano", "agent-jj25nd"]

print(f"\nPublished agents (should be BLOCKED): {PUBLISHED_AGENTS}")
print(f"Unpublished agents (should WORK): {UNPUBLISHED_AGENTS}")

In [ ]:
# Test all agents and compare results
def test_agent(agent_name: str, expected_blocked: bool = False):
    """Test an agent and report if it's blocked or working."""
    try:
        agent_ref = {
            "name": agent_name,
            "version": "1",
            "type": "agent_reference"
        }
        
        resp = openai_client.responses.create(
            input=[{"role": "user", "content": "Say hello in one word."}],
            extra_body={"agent": agent_ref},
        )
        
        # If we get here, agent is working
        status = "✅ WORKS"
        result = resp.output_text[:50] if hasattr(resp, 'output_text') else str(resp.output)[:50]
        
        if expected_blocked:
            status = "⚠️ WORKS (expected BLOCKED)"
            
        return status, result
        
    except Exception as e:
        error_msg = str(e).lower()
        if "blocked" in error_msg or "conditional access" in error_msg or "access denied" in error_msg:
            status = "❌ BLOCKED"
            if expected_blocked:
                status = "✅ BLOCKED (as expected)"
            return status, "Conditional Access policy blocked this agent"
        else:
            return f"⚠️ ERROR", str(e)[:100]

print("=" * 70)
print("TESTING AGENTS WITH CA POLICY ENABLED")
print("=" * 70)
print()

# Test published agents (should be blocked)
print("PUBLISHED AGENTS (Target of CA policy - should be BLOCKED):")
print("-" * 70)
for agent in PUBLISHED_AGENTS:
    status, result = test_agent(agent, expected_blocked=True)
    print(f"  {agent}: {status}")
    print(f"    Response: {result}")
    print()

# Test unpublished agents (should work)
print("UNPUBLISHED AGENTS (Not targeted - should WORK):")
print("-" * 70)
for agent in UNPUBLISHED_AGENTS:
    status, result = test_agent(agent, expected_blocked=False)
    print(f"  {agent}: {status}")
    print(f"    Response: {result}")
    print()

### Enable the Blocking Policy

Now enable the policy (change from Report-only to On):

**Via Entra Admin Center:**
1. Go to **Protection > Conditional Access > Policies**
2. Click on "Block Hosted Agent - Demo"
3. Change **Enable policy** from "Report-only" to **On**
4. Save

**Via Graph API:**

In [ ]:
# Update policy to enabled (blocking) state
# You need the policy ID from when it was created

POLICY_ID = ""  # Fill in your policy ID here
ENABLE_BLOCKING = False  # Set to True to enable blocking

if ENABLE_BLOCKING and POLICY_ID and GRAPH_TOKEN:
    headers = {
        "Authorization": f"Bearer {GRAPH_TOKEN}",
        "Content-Type": "application/json"
    }
    
    # Change state to "enabled" (enforcing)
    resp = requests.patch(
        f"https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies/{POLICY_ID}",
        headers=headers,
        json={"state": "enabled"}
    )
    
    if resp.status_code == 204:
        print("Policy enabled! Agent access is now BLOCKED.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
else:
    print("Policy update skipped.")
    print("To enable blocking: set POLICY_ID and ENABLE_BLOCKING = True")

In [ ]:
# Test again - should now be blocked!
# Wait a minute or two for policy to propagate

import time
print("Waiting 30 seconds for policy propagation...")
time.sleep(30)

try:
    response = client.agents.invoke(
        agent_name=HOSTED_AGENT_NAME,
        content="Hello, what is 2 + 2?"
    )
    print("Agent Response:")
    print(response)
    print("\nNote: If you see a response, the policy may not be enforced yet.")
    
except Exception as e:
    error_msg = str(e).lower()
    if "blocked" in error_msg or "conditional access" in error_msg or "access denied" in error_msg:
        print(">>> SUCCESS: Conditional Access policy BLOCKED the agent! <<<")
        print(f"\nError details: {e}")
    else:
        print(f"Error (may be unrelated to CA): {e}")

---

## Section 5: View Sign-in Logs

Check the Entra sign-in logs to see Conditional Access evaluation.

### Via Entra Admin Center (Manual)

1. Go to [Microsoft Entra Admin Center](https://entra.microsoft.com)
2. Navigate to **Monitoring & health > Sign-in logs**
3. Filter by:
   - **Agent type**: Agent Identity or Agent ID user
   - **Is Agent**: Yes
4. Look for entries from your hosted agent
5. Click on an entry to see:
   - **Conditional Access tab**: Shows which policies were evaluated
   - **Result**: Success, Failure, or Blocked

In [ ]:
# Query sign-in logs via Graph API
# Note: Requires AuditLog.Read.All or Directory.Read.All permission

if GRAPH_TOKEN and AGENT_PRINCIPAL_ID:
    headers = {"Authorization": f"Bearer {GRAPH_TOKEN}"}
    
    # Query service principal sign-ins for our agent
    filter_query = f"servicePrincipalId eq '{AGENT_PRINCIPAL_ID}'"
    
    resp = requests.get(
        f"https://graph.microsoft.com/v1.0/auditLogs/signIns",
        headers=headers,
        params={
            "$filter": filter_query,
            "$top": 10,
            "$orderby": "createdDateTime desc"
        }
    )
    
    if resp.status_code == 200:
        sign_ins = resp.json().get("value", [])
        print(f"Found {len(sign_ins)} recent sign-in events for the agent:")
        for si in sign_ins:
            print(f"\n  Time: {si.get('createdDateTime')}")
            print(f"  Status: {si.get('status', {}).get('errorCode', 'Success')}")
            print(f"  Resource: {si.get('resourceDisplayName')}")
            ca = si.get('conditionalAccessStatus', 'notApplied')
            print(f"  CA Status: {ca}")
    else:
        print(f"Error querying logs: {resp.status_code}")
        print("Note: You may need additional permissions (AuditLog.Read.All)")
else:
    print("Cannot query logs - missing token or principal ID")

---

## Section 6: Capability Hosts and Agent Identity

**Capability Hosts** are Azure resources that provide data storage for agents:
- **Threads** → Stored in CosmosDB (conversation history)
- **Files** → Stored in Azure Storage (uploaded documents)
- **Vectors** → Stored in AI Search (embeddings for RAG)

### How Agent Identity Interacts with Capability Hosts

When your agent accesses these resources, it uses its **managed identity** to authenticate:

```
┌─────────────────────┐         ┌────────────────────────────┐
│   Hosted Agent      │  auth   │   Capability Host          │
│   (Agent Identity)  │────────►│   (CosmosDB/Storage/Search)│
│                     │  RBAC   │                            │
└─────────────────────┘         └────────────────────────────┘
```

| Capability Host | Azure Resource | Required RBAC Role |
|-----------------|----------------|-------------------|
| Threads | CosmosDB | Cosmos DB Data Contributor |
| Files | Azure Storage | Storage Blob Data Contributor |
| Vectors | AI Search | Search Index Data Contributor |

In [ ]:
# Get capability hosts configuration for this project
print("=" * 60)
print("CAPABILITY HOSTS CONFIGURATION")
print("=" * 60)

result = subprocess.run([
    "az", "rest", "--method", "GET",
    "--url", f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_ACCOUNT}/projects/{PROJECT_NAME}/capabilityHosts?api-version=2025-04-01-preview"
], capture_output=True, text=True)

capability_hosts = {}
if result.returncode == 0:
    hosts_data = json.loads(result.stdout)
    hosts = hosts_data.get("value", [])
    
    if hosts:
        for host in hosts:
            host_name = host.get("name", "unknown")
            props = host.get("properties", {})
            
            print(f"\n📦 Capability Host: {host_name}")
            print("-" * 40)
            
            # Show configuration
            storage_config = props.get("storageConfigurations", {})
            
            # Threads (CosmosDB)
            threads = storage_config.get("threadStorage", {})
            if threads:
                cosmosdb_id = threads.get("cosmosDBResourceId", "").split("/")[-1] if threads.get("cosmosDBResourceId") else "N/A"
                print(f"  Threads Storage:    CosmosDB ({cosmosdb_id})")
                capability_hosts["threads"] = threads.get("cosmosDBResourceId")
            
            # Files (Storage)
            files = storage_config.get("fileStorage", {})
            if files:
                storage_id = files.get("storageAccountResourceId", "").split("/")[-1] if files.get("storageAccountResourceId") else "N/A"
                print(f"  Files Storage:      Storage Account ({storage_id})")
                capability_hosts["files"] = files.get("storageAccountResourceId")
            
            # Vectors (AI Search)
            vectors = storage_config.get("vectorStorage", {})
            if vectors:
                search_id = vectors.get("aiSearchResourceId", "").split("/")[-1] if vectors.get("aiSearchResourceId") else "N/A"
                print(f"  Vector Storage:     AI Search ({search_id})")
                capability_hosts["vectors"] = vectors.get("aiSearchResourceId")
                
    else:
        print("No capability hosts configured for this project.")
        print("Agents will use default platform storage.")
else:
    print(f"Error retrieving capability hosts: {result.stderr[:200]}")

In [ ]:
# Show how agent identity maps to capability host access
print("=" * 60)
print("AGENT IDENTITY → CAPABILITY HOST ACCESS")
print("=" * 60)

print("""
When an agent needs to store/retrieve data, the access flow is:

1. Agent makes API call (e.g., create thread, upload file)
2. Foundry validates the request
3. Agent's managed identity authenticates to capability host
4. RBAC permissions are checked
5. Data operation proceeds (or fails if no access)

""")

# Show identity-to-resource mapping
print("Identity to Resource Mapping:")
print("-" * 60)

if SHARED_AGENT_ID:
    print(f"\n🔹 UNPUBLISHED AGENTS (shared identity: {SHARED_AGENT_ID[:20]}...)")
    print("   All unpublished agents authenticate as THIS identity")
    print("   → If identity loses access to CosmosDB, ALL unpublished agents lose thread storage")
    
if PUBLISHED_AGENT_ID:
    print(f"\n🔹 PUBLISHED AGENT 'my-hosted-agent' (identity: {PUBLISHED_AGENT_ID[:20]}...)")
    print("   This agent authenticates as its OWN distinct identity")
    print("   → Losing access only affects this specific agent")

print("\n" + "=" * 60)
print("SECURITY IMPLICATIONS")
print("=" * 60)
print("""
| Scenario | Shared Identity Impact | Distinct Identity Impact |
|----------|------------------------|-------------------------|
| Revoke CosmosDB access | ALL unpublished agents lose threads | Only that agent loses threads |
| Revoke Storage access | ALL unpublished agents lose files | Only that agent loses files |
| CA policy blocks identity | ALL unpublished agents blocked | Only that agent blocked |
""")

In [ ]:
# Check RBAC role assignments for agent identities on capability host resources
print("=" * 60)
print("RBAC ROLE ASSIGNMENTS CHECK")
print("=" * 60)

def check_rbac_for_identity(identity_id, identity_name):
    """Check what RBAC roles an identity has."""
    print(f"\n🔍 Checking roles for: {identity_name}")
    print(f"   Identity ID: {identity_id}")
    
    # Get role assignments for this identity
    result = subprocess.run([
        "az", "role", "assignment", "list",
        "--assignee", identity_id,
        "--all",  # Include inherited
        "-o", "json"
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        assignments = json.loads(result.stdout)
        if assignments:
            print(f"   Found {len(assignments)} role assignment(s):")
            for ra in assignments:
                role = ra.get("roleDefinitionName", "Unknown")
                scope = ra.get("scope", "").split("/")[-1]
                resource_type = ra.get("scope", "").split("/")[-2] if "/" in ra.get("scope", "") else ""
                print(f"     - {role} on {resource_type}/{scope}")
        else:
            print("   No direct role assignments found (may use inherited roles)")
    else:
        print(f"   Could not check roles: {result.stderr[:100]}")

# Check shared identity (unpublished agents)
if SHARED_AGENT_ID:
    try:
        check_rbac_for_identity(SHARED_AGENT_ID, "Shared Project Identity")
    except Exception as e:
        print(f"   Error checking shared identity: {e}")

# Check published agent identity
if PUBLISHED_AGENT_ID:
    try:
        check_rbac_for_identity(PUBLISHED_AGENT_ID, "Published Agent (my-hosted-agent)")
    except Exception as e:
        print(f"   Error checking published identity: {e}")

print("\n" + "-" * 60)
print("Note: Role assignments may be at resource group or subscription level")
print("Check Azure Portal > Resource > Access control (IAM) for full details")

---

## Section 7: Cleanup - Disable/Delete the Policy

After testing, disable or delete the blocking policy to restore agent functionality.

In [ ]:
# Disable the policy (set to report-only)
DISABLE_POLICY = False  # Set to True to disable

if DISABLE_POLICY and POLICY_ID and GRAPH_TOKEN:
    headers = {
        "Authorization": f"Bearer {GRAPH_TOKEN}",
        "Content-Type": "application/json"
    }
    
    resp = requests.patch(
        f"https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies/{POLICY_ID}",
        headers=headers,
        json={"state": "enabledForReportingButNotEnforced"}  # Back to report-only
    )
    
    if resp.status_code == 204:
        print("Policy disabled (report-only). Agent access restored.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
else:
    print("Policy disable skipped.")

In [ ]:
# Or delete the policy entirely
DELETE_POLICY = False  # Set to True to delete

if DELETE_POLICY and POLICY_ID and GRAPH_TOKEN:
    headers = {"Authorization": f"Bearer {GRAPH_TOKEN}"}
    
    resp = requests.delete(
        f"https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies/{POLICY_ID}",
        headers=headers
    )
    
    if resp.status_code == 204:
        print("Policy deleted successfully.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
else:
    print("Policy deletion skipped.")

---

## Summary

### What We Demonstrated (Option B: Per-Agent Control)

| Step | Action | Result |
|------|--------|--------|
| 1 | Found SHARED project identity | All unpublished agents use this |
| 2 | Found DISTINCT published identity | Only `my-hosted-agent` uses this |
| 3 | Created CA policy targeting distinct ID | Blocks only the published agent |
| 4 | Tested both agent types | Published=blocked, Unpublished=works |
| 5 | Explored capability hosts | Agent identity authenticates to storage resources |

### Identity Comparison

| Agent | Type | Identity ID | CA Policy Result |
|-------|------|-------------|------------------|
| `my-hosted-agent` | Published | `066a09ef...` (distinct) | BLOCKED |
| `chatbot-gpt5nano` | Unpublished | `4d5ff6dd...` (shared) | WORKS |
| `agent-jj25nd` | Unpublished | `4d5ff6dd...` (shared) | WORKS |

### Capability Hosts and Identity

| Capability Host | Resource | Agent Identity Role |
|-----------------|----------|---------------------|
| Threads | CosmosDB | Authenticates to read/write conversations |
| Files | Azure Storage | Authenticates to upload/download documents |
| Vectors | AI Search | Authenticates to store/query embeddings |

**Key Insight:** The same identity that CA policies target is also used to authenticate to capability host resources. Revoking access at the RBAC level on these resources provides another control mechanism beyond CA policies.

### Key Takeaways

1. **Publishing gives granular control** - Each published agent can be managed independently
2. **Shared identity = shared fate** - Blocking shared ID affects ALL unpublished agents
3. **CA policies are identity-based** - Target specific agents without affecting others
4. **Capability hosts use same identity** - Agent identity authenticates to storage resources
5. **Audit trail in sign-in logs** - See exactly which policies blocked which agents

### Real-World Use Cases

| Scenario | Implementation |
|----------|---------------|
| **Block compromised agent** | CA policy on that agent's distinct ID |
| **Kill switch for all dev agents** | CA policy on shared project identity |
| **Restrict to specific resources** | CA policy with resource conditions |
| **Geo-restriction** | CA policy with location conditions |
| **Revoke storage access** | Remove RBAC role on capability host resource |

---

## Next Steps

Continue to `../04-foundry-agent-memory` to add memory/state to your hosted agent.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>